# Make a 360 degree star chart for viewing with the Oculus gallery app

In [1]:
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy import signal

In [2]:
star = pd.read_csv("star_data_lots.csv") #Loading the data set

In [3]:
star.head()

,proper,ra,dec,mag,con
0,Sol,0.000000,0.000000,-26.70,NaN
1,Sirius,6.752481,-16.716116,-1.44,CMa
2,Canopus,6.399195,-52.695660,-0.62,Car
3,Arcturus,14.261030,19.182410,-0.05,Boo
4,Rigil Kentaurus,14.660765,-60.833976,-0.01,Cen


In [4]:
star[["ra"]] = star.ra*15
star[["dec"]] = star.dec*-1

In [5]:
star = star[star.proper != "Sol"] ##Removing all rows containg Sun
star.head()

,proper,ra,dec,mag,con
1,Sirius,101.287215,16.716116,-1.44,CMa
2,Canopus,95.987925,52.695660,-0.62,Car
3,Arcturus,213.915450,-19.182410,-0.05,Boo
4,Rigil Kentaurus,219.911475,60.833976,-0.01,Cen
5,Vega,279.234600,-38.783692,0.03,Lyr


In [6]:
star.loc[(star.mag > 5.0 ), "mag"] = 5.0

In [7]:
star.mag.min()

-1.44

In [8]:
rate = (255 - 20) / (-1.44 - 5.0)
offset = 20 - (5.0 * rate)

In [9]:
#Mapping the intensity values of the stars to fit within the color spectrum understood by python
star[["mag"]] = round((star.mag * rate) + offset).astype(int) 

In [10]:
star.head()

,proper,ra,dec,mag,con
1,Sirius,101.287215,16.716116,255,CMa
2,Canopus,95.987925,52.695660,225,Car
3,Arcturus,213.915450,-19.182410,204,Boo
4,Rigil Kentaurus,219.911475,60.833976,203,Cen
5,Vega,279.234600,-38.783692,201,Lyr


In [11]:
background_image = np.zeros((2048, 4096), np.uint8) #Creating a black image of zero vectors

In [12]:
rate_1 = (4096 - 0) / (360 - 0)

In [13]:
star[["ra"]] = round(star.ra * rate_1).astype(int) #Mapping the right ascension values to fillup the pixels of the image

In [14]:
star.head()

,proper,ra,dec,mag,con
1,Sirius,1152,16.716116,255,CMa
2,Canopus,1092,52.695660,225,Car
3,Arcturus,2434,-19.182410,204,Boo
4,Rigil Kentaurus,2502,60.833976,203,Cen
5,Vega,3177,-38.783692,201,Lyr


In [15]:
rate_2 = (2048 - 0)/(180)
offset_2 = 0 - (-90*rate_2)

In [16]:
star[["dec"]] = round((star.dec* rate_2) + offset_2).astype(int)
star.head()

,proper,ra,dec,mag,con
1,Sirius,1152,1214,255,CMa
2,Canopus,1092,1624,225,Car
3,Arcturus,2434,806,204,Boo
4,Rigil Kentaurus,2502,1716,203,Cen
5,Vega,3177,583,201,Lyr


In [17]:
longitudes = list(star.ra)
latitudes = list(star.dec)
pixels = list(star.mag)

In [18]:
background_image.shape

(2048, 4096)

In [19]:
#Mapping the star positions to the black background image
for i in range(star.shape[0]):
    if (longitudes[i] < background_image.shape[1]) and (latitudes[i] < background_image.shape[0]):
        background_image[(latitudes[i])][(longitudes[i])] = pixels[i]

In [20]:
#Convolution of final image to blur the stars and simulate the atmosphere
convK = np.array([(0.0, 0.5, 0.0), (0.5, 1.0, 0.5), (0.0, 0.5, 0.0)], np.uint8)
imdat = signal.convolve2d(background_image, convK, 'same', 'fill', fillvalue = 0)

In [21]:
Image.fromarray(imdat).save('star.jpg', quality=95)